In a previous life I used to play poker for a living and in the early I used a tool called PokerStove a lot to understand all in pre-flop probabilities. 

After exploring the tool for a while you start to get a rough idea of your odds of certain kinds of hands, e.g. a pair vs two overcards (~50/50), a dominated hand (~70/30), a pair vs an overpair (~80/20) and so on.

I think part of what makes computers interesting is the ability to use computation to solve problems we couldn't easily solve by hand or with other tools. This notebook is a study in calculating probabilities of poker hands preflop from first principles.

I find when starting an exploration having a concrete goal in mind is useful to get started, so for this study I want to calculate the probabilities of a 3-way all in between `2c2d`, `9h10h` and `AcKs`.

My thinking is that we need a way to calculate the rank of the strongest 5 cards of a 7 card hand (2 hole cards + 5 from the board). Once we can do this we start with the above hands drawn, shuffle the rest of the deck and repeatedly draw the 5 card board and calculate the who wins/ties/loses and then tally them up over a number of iterations (technically called Monte Carlo simulation?).

To start we need to a representation of a card, a hand and a deck. We've got a bunch of choices here:
1. A Card is represented by object consisting of a Rank (e.g. ACE, JACK, TWO) and a Suit (e.g. HEARTS, CLUBS). A Hand is an array of Cards, and a Deck is a array of Cards
2. A Card is represented by a number from 0-53. A Hand is an Array of Cards. A Deck is an array of Cards
3. A Card is represented as a bit in a bitset which represents a Hand. A full bitset or an array of single bit bitsets could represent a Deck.

These representations have different pros and cons but for now I'm going to start with option 2

In [24]:
from enum import IntEnum, auto

class Rank(IntEnum):
    ACE = 0
    TWO = auto()
    THREE = auto()
    FOUR = auto()
    FIVE = auto()
    SIX = auto()
    SEVEN = auto()
    EIGHT = auto()
    NINE = auto()
    TEN = auto()
    JACK = auto()
    QUEEN = auto()
    KING = auto()


In [36]:

class Suit(IntEnum):
    CLUBS = 0
    DIAMONDS = auto()
    HEARTS = auto()
    SPADES = auto()

In [37]:
from enum import IntEnum, auto

class Card(IntEnum):
    ACE_CLUBS = 0
    TWO_CLUBS = auto()
    THREE_CLUBS = auto()
    FOUR_CLUBS = auto()
    FIVE_CLUBS = auto()
    SIX_CLUBS = auto()
    SEVEN_CLUBS = auto()
    EIGHT_CLUBS = auto()
    NINE_CLUBS = auto()
    TEN_CLUBS = auto()
    JACK_CLUBS = auto()
    QUEEN_CLUBS = auto()
    KING_CLUBS = auto()
    ACE_DIAMONDS = auto()
    TWO_DIAMONDS = auto()
    THREE_DIAMONDS = auto()
    FOUR_DIAMONDS = auto()
    FIVE_DIAMONDS = auto()
    SIX_DIAMONDS = auto()
    SEVEN_DIAMONDS = auto()
    EIGHT_DIAMONDS = auto()
    NINE_DIAMONDS = auto()
    TEN_DIAMONDS = auto()
    JACK_DIAMONDS = auto()
    QUEEN_DIAMONDS = auto()
    KING_DIAMONDS = auto()
    ACE_HEARTS = auto()
    TWO_HEARTS = auto()
    THREE_HEARTS = auto()
    FOUR_HEARTS = auto()
    FIVE_HEARTS = auto()
    SIX_HEARTS = auto()
    SEVEN_HEARTS = auto()
    EIGHT_HEARTS = auto()
    NINE_HEARTS = auto()
    TEN_HEARTS = auto()
    JACK_HEARTS = auto()
    QUEEN_HEARTS = auto()
    KING_HEARTS = auto()
    ACE_SPADES = auto()
    TWO_SPADES = auto()
    THREE_SPADES = auto()
    FOUR_SPADES = auto()
    FIVE_SPADES = auto()
    SIX_SPADES = auto()
    SEVEN_SPADES = auto()
    EIGHT_SPADES = auto()
    NINE_SPADES = auto()
    TEN_SPADES = auto()
    JACK_SPADES = auto()
    QUEEN_SPADES = auto()
    KING_SPADES = auto()

    def rank(self):
        return Rank(self.value % 13)
    
    def suit(self):
        return Suit(self.value % 4)
    
# Interestingly we could also generate the enum like this:
# Card = IntEnum('Card', [f"{rank}_{suit}" for rank, suit in itertools.product(Rank.__members__, Suit.__members__)])

In [17]:
[Card(i) for i in range(52)]

0

In [41]:
def test_rank():
    assert(Card.ACE_CLUBS.rank() == Card.ACE_DIAMONDS.rank())
    assert(Card.ACE_DIAMONDS.rank() == Card.ACE_HEARTS.rank())
    assert(Card.ACE_HEARTS.rank() == Card.ACE_SPADES.rank())

    assert(Card.KING_CLUBS.rank() == Card.KING_DIAMONDS.rank())
    assert(Card.KING_DIAMONDS.rank() == Card.KING_HEARTS.rank())
    assert(Card.KING_HEARTS.rank() == Card.KING_SPADES.rank())

test_rank()

In [45]:
def test_suit():
    assert(Card.ACE_CLUBS.suit() == Card.KING_CLUBS.suit())
    assert(Card.ACE_DIAMONDS.suit() == Card.KING_DIAMONDS.suit())
    assert(Card.ACE_HEARTS.suit() == Card.KING_HEARTS.suit())
    assert(Card.ACE_SPADES.suit() == Card.KING_SPADES.suit())

test_suit()

So now we have a way to represent a Rank, Suit, Card, Hand and Deck. What we need next is a way to calculate the HandRank. In poker the hand ranks are as follows:
1. Straight flush
2. Four of a kind
3. Full house
4. Flush
5. Straight
6. Three of a kind
7. Two pair
8. One pair
9. High card

In [ ]:
hand = [Card.ACE_CLUBS, Card.JACK_CLUBS, Card.QUEEN_CLUBS, Card.TWO_CLUBS, Card.QUEEN_CLUBS, Card.ACE_DIAMONDS, Card.TWO_HEARTS]